In [34]:
import pandas as pd
import sqlite3
import os
from datetime import datetime, timedelta

In [35]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [36]:
# Load the data
sensor_dataset = "data/pedestrian-counting-system-sensor-locations.csv"
sensors_df = pd.read_csv(os.path.join(os.getcwd(), sensor_dataset))

counts_dataset = "data/pedestrian-counting-system-past-hour-counts-per-minute.csv"
counts_df = pd.read_csv(os.path.join(os.getcwd(), counts_dataset))

In [37]:
# Checking the column names
print(f"Columns in sensors_df:{sensors_df.columns.values}")
print(f"Columns in counts_df:{counts_df.columns.values}")

Columns in sensors_df:['location_id' 'Sensor_description' 'sensor_name' 'Installation_date'
 'Note' 'location_type' 'Status' 'Direction_1' 'Direction_2' 'Latitude'
 'Longitude' 'Location']
Columns in counts_df:['LocationID' 'SensingDateTime' 'SensingDate' 'SensingTime' 'Direction_1'
 'Direction_2' 'total_of_Directions']


In [38]:
print(sensors_df.dtypes)

location_id             int64
Sensor_description     object
sensor_name            object
Installation_date      object
Note                   object
location_type          object
Status                 object
Direction_1            object
Direction_2            object
Latitude              float64
Longitude             float64
Location               object
dtype: object


In [39]:
# Merging two tables using location_id
merged_df = pd.merge(
    sensors_df, counts_df, left_on="location_id", right_on="LocationID"
)

merged_df["dt"] = pd.to_datetime(merged_df["Installation_date"])

merged_df[["day_of_week", "year", "month", "date", "hour"]] = merged_df["dt"].apply(
    lambda x: pd.Series([x.day_name(), x.year, x.month, x.date(), x.hour])
)

In [40]:
# Creating the Databse
sensors_db = sqlite3.connect("sensors_db.db")
counts_db = sqlite3.connect("counts_db.db")

sensors_df.to_sql("sensors", sensors_db, if_exists="replace", index=False)
counts_df.to_sql("counts", counts_db, if_exists="replace", index=False)

25446

In [41]:
# Checking the inserted data
sensors_data = pd.read_sql_query("SELECT * from sensors", sensors_db)
sensors_data.head()

,location_id,Sensor_description,sensor_name,Installation_date,Note,location_type,Status,Direction_1,Direction_2,Latitude,Longitude,Location
0,1,Bourke Street Mall (North),Bou292_T,2009-03-24,None,Outdoor,A,East,West,-37.813494,144.965153,"-37.81349441, 144.96515323"
1,3,Melbourne Central,Swa295_T,2009-03-25,None,Outdoor,A,North,South,-37.811015,144.964295,"-37.81101524, 144.96429485"
2,7,Birrarung Marr,FedPed_T,2014-12-17,None,Outdoor,A,East,West,-37.818629,144.971694,"-37.81862929, 144.97169395"
3,9,Southern Cross Station,Col700_T,2009-03-23,None,Outdoor,A,East,West,-37.819830,144.951026,"-37.81982992, 144.95102555"
4,12,New Quay,NewQ_T,2009-01-21,None,Outdoor,A,East,West,-37.814580,144.942924,"-37.81457988, 144.94292398"


In [43]:
# Checking the inserted data
counts_data = pd.read_sql_query("SELECT * from counts", counts_db)
counts_data.head()

,LocationID,SensingDateTime,SensingDate,SensingTime,Direction_1,Direction_2,total_of_Directions
0,6,2023-06-05T11:26:00+10:00,2023-06-05,11:26,6,8,14
1,29,2023-06-05T11:26:00+10:00,2023-06-05,11:26,2,1,3
2,31,2023-06-05T11:26:00+10:00,2023-06-05,11:26,0,1,1
3,36,2023-06-05T11:26:00+10:00,2023-06-05,11:26,5,7,12
4,37,2023-06-05T11:26:00+10:00,2023-06-05,11:26,1,3,4


In [ ]:
# Decline in the last 3 years
current_year = merged_df["year"].max()
two_years_ago = current_year - 3

last_two_years = merged_df[
    (merged_df["year"] >= two_years_ago) & (merged_df["year"] <= current_year)
]

start_date = datetime.strptime("2020-03-23", "%Y-%m-%d").date()
end_date = datetime.strptime("2022-08-06", "%Y-%m-%d").date()

lockdown_dates = []
current_date = start_date

while current_date <= end_date:
    lockdown_dates.append(current_date)
    current_date += timedelta(days=1)

lockdown_df = last_two_years[last_two_years["date"].isin(lockdown_dates)]

lockdown_counts = (
    lockdown_df.groupby(["Sensor_description"])["total_of_Directions"]
    .sum()
    .reset_index()
)

total_counts = (
    last_two_years.groupby(["Sensor_description"])["total_of_Directions"]
    .sum()
    .reset_index()
)

combined_counts = pd.merge(total_counts, lockdown_counts, on="Sensor_description")

combined_counts["percent_change"] = (
    combined_counts["total_of_Directions_x"] - combined_counts["total_of_Directions_y"]
) / combined_counts["total_of_Directions_x"]

most_decline = combined_counts.loc[
    combined_counts["percent_change"].idxmax(), "Sensor_description"
]

In [ ]:
# Most growth in the last year
last_year = merged_df[merged_df["year"] == current_year - 1]
growth_df = (
    last_year.groupby(["Sensor_description"])["total_of_Directions"]
    .sum()
    .reset_index()
    .sort_values(by="total_of_Directions", ascending=False)
)

most_growth = growth_df.loc[
    growth_df["total_of_Directions"].idxmax(), "Sensor_description"
]
most_growth_count = growth_df.loc[
    growth_df["total_of_Directions"].idxmax(), "total_of_Directions"
]

In [52]:
# Average pedestrans count in location hourly
avg_hourly_counts_loc = (
    merged_df.groupby(["Sensor_description", "hour"])["total_of_Directions"]
    .mean()
    .reset_index()
    .sort_values("total_of_Directions", ascending=False)
)
print("Average pedestrans count in location hourly")
avg_hourly_counts_loc.head(10)

Average pedestrans count in location hourly


,Sensor_description,hour,total_of_Directions
15,Elizabeth St - Flinders St (East) - New footpath,0,40.708333
9,Building 80 RMIT,0,34.147727
33,La Trobe St- William St (South),0,18.771186
22,Flinders La-Swanston St (West),0,16.187175
59,Spencer St-Collins St (North),0,13.822142
1,114 Flinders Street Car Park Footpath,0,12.475728
58,Southern Cross Station,0,12.255754
21,Flagstaff station (East),0,10.735294
67,Town Hall (West),0,10.116601
72,William St - Little Lonsdale St (West),0,9.631068


In [ ]:
# Busiest day in the week
total_counts_dow = (
    merged_df.groupby(["day_of_week"])["total_of_Directions"]
    .sum()
    .reset_index()
    .sort_values("total_of_Directions", ascending=False)
)
print("Busiest day in the week")
total_counts_dow

Busiest day in the week


,day_of_week,total_of_Directions
1,Monday,51792
3,Thursday,36263
5,Wednesday,26671
0,Friday,18504
4,Tuesday,12986
2,Saturday,9793


In [ ]:
# Busiest month in the week
total_counts_month = (
    merged_df.groupby(["month"])["total_of_Directions"]
    .sum()
    .reset_index()
    .sort_values("total_of_Directions", ascending=False)
)
print("Busiest month in the week:")
total_counts_month

Busiest month in the week:


,month,total_of_Directions
2,3,40876
8,9,28754
5,6,19941
9,10,12528
7,8,10967
3,4,10199
11,12,9635
6,7,8303
0,1,5809
10,11,4362


In [ ]:
# Average hourly pedestrian counts by day
avg_hourly_counts_dow = (
    merged_df.groupby(["day_of_week", "hour"])["total_of_Directions"]
    .mean()
    .reset_index()
    .sort_values("total_of_Directions", ascending=False)
)
print("Average hourly pedestrian counts by day")
avg_hourly_counts_dow

Average hourly pedestrian counts by day


,day_of_week,hour,total_of_Directions
3,Thursday,0,7.066056
2,Saturday,0,7.050396
1,Monday,0,6.714026
0,Friday,0,5.868696
5,Wednesday,0,5.299225
4,Tuesday,0,4.292893


In [ ]:
# Average hourly pedestrian counts by month
avg_hourly_counts_month = (
    merged_df.groupby(["month", "hour"])["total_of_Directions"].mean().reset_index().sort_values("total_of_Directions", ascending=False)
)
print("Average hourly pedestrian counts by month:")
avg_hourly_counts_month

Average hourly pedestrian counts by month:


,month,hour,total_of_Directions
11,12,0,7.652899
1,2,0,7.459813
5,6,0,7.206722
2,3,0,7.162432
7,8,0,6.574940
8,9,0,6.347461
3,4,0,5.641040
6,7,0,5.038228
9,10,0,4.999202
0,1,0,4.011740


In [49]:
# Top 10 (most pedestrians) locations by day
top10_day = merged_df.groupby(["date", "Sensor_description"])["total_of_Directions"].sum().reset_index().sort_values(by="total_of_Directions", ascending=False).groupby("date").head(10)


print("Top 10 (most pedestrians) locations by day:")
top10_day.head(10)

Top 10 (most pedestrians) locations by day:


,date,Sensor_description,total_of_Directions
32,2017-06-29,Flinders La-Swanston St (West),9340
18,2013-09-02,Spencer St-Collins St (North),7616
64,2021-12-03,Elizabeth St - Flinders St (East) - New footpath,5862
3,2009-03-23,Town Hall (West),5119
7,2009-03-25,Flinders Street Station Underpass,5018
45,2018-09-27,Bourke St - Spencer St (North),4906
31,2016-04-11,Southbank,4854
42,2018-07-19,Elizabeth St-La Trobe St (East),4820
2,2009-03-23,Southern Cross Station,4792
36,2017-08-24,Melbourne Central-Elizabeth St (East),4139


In [47]:
# Top 10 (most pedestrians) locations by month
top10_month = merged_df.groupby(["year", "month", "Sensor_description"])["total_of_Directions"].sum().reset_index().sort_values(by="total_of_Directions", ascending=False).groupby(["year", "month"]).head(10)

print("Top 10 (most pedestrians) locations by month:")
top10_month.head(10)

Top 10 (most pedestrians) locations by month:


,year,month,Sensor_description,total_of_Directions
32,2017,6,Flinders La-Swanston St (West),9340
20,2013,9,Spencer St-Collins St (North),7616
64,2021,12,Elizabeth St - Flinders St (East) - New footpath,5862
11,2009,3,Town Hall (West),5119
6,2009,3,Flinders Street Station Underpass,5018
45,2018,9,Bourke St - Spencer St (North),4906
31,2016,4,Southbank,4854
42,2018,7,Elizabeth St-La Trobe St (East),4820
10,2009,3,Southern Cross Station,4792
36,2017,8,Melbourne Central-Elizabeth St (East),4139


In [50]:
print(f"Location with the most decline due to lockdowns in the last 3 years: {most_decline}")

Location with the most decline due to lockdowns in the last 3 years: Elizabeth St - Flinders St (East) - New footpath
